In [11]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/addbs_adascog_testingdict'

# 01 - Generate the Correlation Maps for Training

Enter the path to the dictionary generated by Notebook 01

In [12]:
dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/training_dict_memory.json'

What correlation method would you like to use?
- Pearson is very fast, but depends on linearity and is sensitive to outliers
- Spearman is slower, but much more robust

In [13]:
correlation = 'spearman'

In [14]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
from calvin_utils.ccm_utils.stat_utils import CorrelationCalculator
data_loader = DataLoader(dict_path)
correlation_calculator = CorrelationCalculator(method=correlation, verbose=False)
corr_map_dict = correlation_calculator.generate_correlation_maps(data_loader)

Evaluating ANT DBS SANTE
Evaluating STN DBS
Evaluating ANT DBS Maastricht
Evaluating Epilepsy iEEG
Evaluating Epilepsy Hypometabolism
Evaluating Acute Stroke
Evaluating Multiple Sclerosis
Evaluating Penetrating TBI
Evaluating Parietal TMS Hebscher
Evaluating Parietal TMS Wang
Evaluating Parietal TMS Hermiller 2019a
Evaluating Parietal TMS Hermiller 2019b


# 02 - Get the Data for Testing

In [15]:
testing_dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/addbs_adascog_testingdict.json'

In [16]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
test_data_loader = DataLoader(testing_dict_path)

# 03 - Run Optimizer

Generate the Convergent Maps and Save the Individual Correlation Maps

In [17]:
from calvin_utils.ccm_utils.optimization import NiftiOptimizer
optimizer = NiftiOptimizer(corr_map_dict, test_data_loader, load_in_time=False)
optimal_map = optimizer.optimize(store_iters=False, store_best=True)
print(optimizer.W)

Removing NaNS from self.coor_map_dict.
===Imaging Optimizer Initialized===
Initializing: 
	 Weights: (1, 12)  
	 Training Maps: (12, 228483)
===        Training Data        ===
	 Training Dataset (For weighted average): ANT DBS SANTE
	 Training Dataset (For weighted average): STN DBS
	 Training Dataset (For weighted average): ANT DBS Maastricht
	 Training Dataset (For weighted average): Epilepsy iEEG
	 Training Dataset (For weighted average): Epilepsy Hypometabolism
	 Training Dataset (For weighted average): Acute Stroke
	 Training Dataset (For weighted average): Multiple Sclerosis
	 Training Dataset (For weighted average): Penetrating TBI
	 Training Dataset (For weighted average): Parietal TMS Hebscher
	 Training Dataset (For weighted average): Parietal TMS Wang
	 Training Dataset (For weighted average): Parietal TMS Hermiller 2019a
	 Training Dataset (For weighted average): Parietal TMS Hermiller 2019b
===         Testing Data        ===
	 Testing Dataset (To predict outcomes using w

Loss: 0, 0/500: 0it [00:00, ?it/s]
Loss: 0.40450467108358873, 1/500: 0it [00:00, ?it/s]
Loss: 0.5236482364556418, 2/500: 0it [00:00, ?it/s]
Loss: 0.5406276709411179, 3/500: 0it [00:00, ?it/s]
Loss: 0.5433587058118008, 4/500: 0it [00:00, ?it/s]
Loss: 0.5535915015617275, 5/500: 0it [00:00, ?it/s]
Loss: 0.5556487461902304, 6/500: 0it [00:00, ?it/s]
Loss: 0.5617203838844393, 7/500: 0it [00:00, ?it/s]
Loss: 0.5909351842046311, 8/500: 0it [00:00, ?it/s]
Loss: 0.6081150169987061, 9/500: 0it [00:00, ?it/s]
Loss: 0.6064901242695534, 10/500: 0it [00:00, ?it/s]
Loss: 0.6053704805578126, 11/500: 0it [00:00, ?it/s]
Loss: 0.6109986533168609, 12/500: 0it [00:00, ?it/s]
Loss: 0.6113014026888043, 13/500: 0it [00:00, ?it/s]
Loss: 0.6100389557532021, 14/500: 0it [00:00, ?it/s]
Loss: 0.616425392640583, 15/500: 0it [00:00, ?it/s]
Loss: 0.619469487166747, 16/500: 0it [00:00, ?it/s]
Loss: 0.6200665663291296, 17/500: 0it [00:00, ?it/s]
Loss: 0.6194197045904488, 18/500: 0it [00:00, ?it/s]
Loss: 0.6168754660006

Converged. L1 norm below thresold: 0.0 < 1e-06


Loss: 0.6310106321827216, 25/500: 0it [00:00, ?it/s]

[[ 0.03150723 -0.00065379 -0.10302156  0.10582227  0.1215108   0.08309791
  -0.00188967  0.02773528  0.01842283  0.13777335  0.14610597  0.22245932]]


Save Final Map

In [18]:
from calvin_utils.ccm_utils.ccm_utils import save_individual_r_maps
save_individual_r_maps(data=optimal_map, out_dir=out_dir, filename='optimized_convergent_map_converged.nii.gz', mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii', verbose=True)

Save Best Map
- Must set store_best=True

In [19]:
best_map = optimizer._converge_maps(W=optimizer.best_weights)
from calvin_utils.ccm_utils.ccm_utils import save_individual_r_maps
save_individual_r_maps(data=optimal_map, out_dir=out_dir, filename='optimized_convergent_map_best.nii.gz', mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii', verbose=True)

Save All Intermediate Maps
- Must have set store_iters=True

In [20]:
for i, vol in enumerate(optimizer.iter_maps):
    save_individual_r_maps(data=optimal_map, 
                           out_dir=out_dir, 
                           filename=f"iteration_maps/iter_{i:03d}.nii.gz", 
                           mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii',
                           verbose=False)

Thats all there is to it

-Calvin